# PySimpleGUI Timer
- PySimpleGUI を使ってタイマーウィジェットを作成
- ポモドーロタイマー(25分,5分)を作る
- Clock と合体させる

## 課題
- カウントダウンはできた
- 設定時間後に休憩時間のカウントダウンについて未実装(済)
- STOP 後に再実行すると BREAK_TIME の文字になってしまう(済)
- adjust_time が STOP するたびに引かれるため、タイマーがずれる
- 別案でおおむね完成したのでもう少し見直したい

[countdown-timer](https://docs.pysimplegui.com/en/latest/cookbook/ecookbook/tools_utilities/countdown-timer/)

[Python タイマーを作ってみた GUI編](https://fuji-pocketbook.net/timer-gui/)

In [21]:
import PySimpleGUI as sg


layout = [
    [sg.Text('タイマーをセットします', size=(None, 2))],
    [sg.Spin([m for m in range(61)]), sg.T('分'), sg.Spin([s for s in range(60)]), sg.T('秒')],
    [sg.VPush()],
    # [sg.Checkbox('デフォルトとして設定する', key='-DEFAULT-')],
    [sg.OK(), sg.Cancel()]]
    

window = sg.Window('セットタイマー', layout)

event, values = window.read()
if event == sg.WIN_CLOSED or event == 'Cancel':
    user = (25, 0)
elif event == 'OK':
    user  = (values[0], values[1])
    # if values['-DEFAULT-']:
    #     with open(path, 'w', encoding='utf-8') as f:
    #         f.write(f'{values[0]} {values[1]}')

window.close()

In [16]:
import time
import PySimpleGUI as sg

FOCUS_M, FOCUS_S = (0, 5)
BREAK_M, BREAK_S = (0, 3)
TOTAL_M, TOTAL_S = (FOCUS_M+BREAK_M, FOCUS_S+BREAK_S)
current_m, current_s = TOTAL_M, TOTAL_S


layout = [
    [sg.Button(f'{FOCUS_M:02} : {FOCUS_S:02}', font=('impact', 35), key='-TIMER_BTN-')],
    [sg.Text(f'FOCUS TIME', font=('impact', 15), key='-MSG-'), sg.Button('Exit')],
    ]

window = sg.Window('Pomodoro Timer', layout, location=(200, 200), element_justification='center')


count_down = False
while True:
    event, values = window.read(timeout=100)

    # event manager
    if event == sg.WIN_CLOSED or event == 'Exit':
        break

    # if count_down and event == '-TIMER_BTN-':
    #         count_down = False
    #         msg = 'STOP'
    #         window['-MSG-'].update(f'{msg}')
    # elif not count_down and event == '-TIMER_BTN-':
    #         count_down = True
    #         msg = 'FOCUS'
    #         end_time = time.time() + current_m * 60 + current_s

    if event == '-TIMER_BTN-':
        if count_down:
            count_down = False
            msg = 'STOP'
            window['-MSG-'].update(f'{msg}')
        elif not count_down:
            count_down = True
            msg = 'FOCUS'
            end_time = time.time() + current_m * 60 + current_s


    # display manager
    if count_down:
        
        remaining_time = end_time - time.time()
        adjust_time = BREAK_M * 60 + BREAK_S

        if remaining_time > adjust_time:
            current_m, current_s = divmod(remaining_time-adjust_time, 60)
        elif 0 <= remaining_time <= adjust_time:
            msg = 'BREAK TIME'
            current_m, current_s = divmod(remaining_time, 60)
        elif remaining_time < 0:
            count_down = False
            msg = 'FINISHED'
            current_m, current_s = FOCUS_M, FOCUS_S
        
        window['-TIMER_BTN-'].update(f'{int(current_m):02} : {int(current_s):02}')
        window['-MSG-'].update(f'{msg}')

window.close()

# time.sleep() をつかう
- [【Python】ポモドーロタイマー：コンソールに作業と休憩時間を入力する](https://anotools.com/python/2945/)

In [5]:
import datetime as dt
import time


FOCUS_M, FOCUS_S = (0, 3)
BREAK_M, BREAK_S = (0, 2)

today = dt.datetime.today()
focus_time = dt.datetime.combine(today, dt.time(0, FOCUS_M, FOCUS_S))
break_time = dt.datetime.combine(today, dt.time(0, BREAK_M, BREAK_S))

focus_start = True
while True:
    if focus_start:
        focus_time_str = focus_time.strftime("%M:%S")
        if focus_time_str == "59:59":
            focus_start = False
        else:
            display_text = f'FOCUS : {focus_time_str}'
            print(display_text)
            focus_time -= dt.timedelta(seconds=1)
            time.sleep(1)
    else:
        break_time_str = break_time.strftime("%M:%S")
        if break_time_str == "59:59":
            break
        else:
            display_text = f'BREAK : {break_time_str}'
            print(display_text)
            break_time -= dt.timedelta(seconds=1)
            time.sleep(1)



FOCUS : 00:03
FOCUS : 00:02
FOCUS : 00:01
FOCUS : 00:00
BREAK : 00:02
BREAK : 00:01
BREAK : 00:00


In [1]:
import PySimpleGUI as sg
import datetime as dt
import time


FOCUS_M, FOCUS_S = (0, 3)
BREAK_M, BREAK_S = (0, 2)

today = dt.datetime.today()
focus_time = dt.datetime.combine(today, dt.time(0, FOCUS_M, FOCUS_S))
focus_time_str = focus_time.strftime("%M:%S")
break_time = dt.datetime.combine(today, dt.time(0, BREAK_M, BREAK_S))

layout = [
    [sg.Button(f'{focus_time_str}', font=('impact', 35), key='-TIMER_BTN-')],
    [sg.Text(f'FOCUS TIME', font=('impact', 15), key='-MSG-'), sg.Button('Exit')],
    ]

window = sg.Window('Pomodoro Timer', layout, location=(200, 200), element_justification='center')


timer_start = False
focus_start = True

while True:
    event, values = window.read(timeout=100)
    if event == sg.WIN_CLOSED or event == 'Exit':
        break

    if event == '-TIMER_BTN-':
        timer_start = True
    
    if timer_start:
        if focus_start:
            focus_time_str = focus_time.strftime("%M:%S")
            if focus_time_str == "59:59":
                focus_start = False
            else:
                # display_text = f'FOCUS : {focus_time_str}'
                # print(display_text)
                window['-TIMER_BTN-'].update(focus_time_str)
                window['-MSG-'].update('FOCUS TIME')
                focus_time -= dt.timedelta(seconds=1)
                time.sleep(1)
        else:
            break_time_str = break_time.strftime("%M:%S")
            if break_time_str == "59:59":
                break
            else:
                # display_text = f'BREAK : {break_time_str}'
                # print(display_text)
                window['-TIMER_BTN-'].update(break_time_str)
                window['-MSG-'].update('BREAK TIME')
                break_time -= dt.timedelta(seconds=1)
                time.sleep(1)


window.close()

In [22]:
import PySimpleGUI as sg
import time

# タイマーの設定
WORK_MINUTES = 25  # 作業時間（分）
BREAK_MINUTES = 5  # 休憩時間（分）

# フォーマットするための関数
def format_time(seconds):
    mins, secs = divmod(seconds, 60)
    return f'{mins:02}:{secs:02}'

# ウィンドウレイアウトの定義
layout = [
    [sg.Text('Pomodoro Timer', font=('Helvetica', 24), justification='center')],
    [sg.Text('', size=(8, 2), font=('Helvetica', 48), justification='center', key='-TIMER-')],
    [sg.Button('Start Work', size=(10, 1)), sg.Button('Start Break', size=(10, 1)), sg.Button('Reset', size=(10, 1))],
    [sg.Button('Exit', size=(10, 1))]
]

# ウィンドウの作成
window = sg.Window('Pomodoro Timer', layout, element_justification='center')

timer_running = False
work_time = WORK_MINUTES * 60  # 作業時間を秒に変換
break_time = BREAK_MINUTES * 60  # 休憩時間を秒に変換
current_time = work_time  # 初期値は作業時間
is_working = True  # 最初は作業状態

# イベントループ
while True:
    event, values = window.read(timeout=1000)  # 1秒ごとに更新

    if event == sg.WIN_CLOSED or event == 'Exit':
        break

    if event == 'Start Work':
        timer_running = True
        current_time = work_time
        is_working = True

    if event == 'Start Break':
        timer_running = True
        current_time = break_time
        is_working = False

    if event == 'Reset':
        timer_running = False
        current_time = work_time if is_working else break_time
        window['-TIMER-'].update(format_time(current_time))

    if timer_running:
        if current_time > 0:
            current_time -= 1
            window['-TIMER-'].update(format_time(current_time))
        else:
            # タイマー終了後の処理
            timer_running = False
            sg.popup('Time is up!', keep_on_top=True)
            current_time = work_time if is_working else break_time

window.close()


## ポモドーロタイマー完成
- 一応考えているようにできたと思う
- 集中時間から作業時間の自動移行
- リセットボタンが欲しい

In [2]:
import PySimpleGUI as sg
import datetime as dt

# 作業時間と休憩時間の設定
FOCUS_M, FOCUS_S = (0, 5)  # 作業時間（分、秒）
BREAK_M, BREAK_S = (0, 3)   # 休憩時間（分、秒）

# タイマーの初期化
def get_time(minutes, seconds):
    today = dt.datetime.today()
    return dt.datetime.combine(today, dt.time(0, minutes, seconds))

# タイマーのリセット機能
def reset_timer():
    focus_time = get_time(FOCUS_M, FOCUS_S)
    break_time = get_time(BREAK_M, BREAK_S)
    return focus_time, break_time

focus_time, break_time = reset_timer()

layout = [
    [sg.Button(f'{focus_time.strftime("%M:%S")}', font=('impact', 35), key='-TIMER_BTN-', size=(6, 1))],
    [sg.Text('READY', font=('impact', 15), key='-MSG-'), sg.Button('Reset')],
]

window = sg.Window('Pomodoro Timer', layout, size=(200, 150), location=(200, 200), element_justification='center')

timer_start = False
focus_start = True  # 最初は作業から開始

# イベントループ
while True:
    event, values = window.read(timeout=1000)  # 1秒ごとに更新
    if event == sg.WIN_CLOSED:
        break

    if event == 'Reset':
        timer_start = False
        focus_start = True
        focus_time, break_time = reset_timer()
        window['-TIMER_BTN-'].update(f'{focus_time.strftime("%M:%S")}')  # ボタンの表示も初期化
        window['-MSG-'].update('READY')  # メッセージも初期化

    if event == '-TIMER_BTN-':
        timer_start = not timer_start  # ボタンでスタート・ストップの切り替え

    if timer_start:
        if focus_start:
            focus_time_str = focus_time.strftime("%M:%S")
            if focus_time_str == "59:59":
                focus_start = False  # 作業時間が終了したら休憩に切り替え
                focus_time = get_time(FOCUS_M, FOCUS_S)  # 作業時間をリセット
            else:
                window['-TIMER_BTN-'].update(focus_time_str)
                window['-MSG-'].update('FOCUS TIME')
                focus_time -= dt.timedelta(seconds=1)
        else:
            break_time_str = break_time.strftime("%M:%S")
            if break_time_str == "59:59":
                timer_start = False  # 休憩時間が終了したらタイマーを停止して初期化
                focus_start = True  # 初期化して作業に切り替え
                # window['-MSG-'].update('TIMER RESETTING...')
                focus_time, break_time = reset_timer()  # タイマーを初期状態にリセット
                window['-TIMER_BTN-'].update(f'{focus_time.strftime("%M:%S")}')  # ボタンの表示も初期化
                window['-MSG-'].update('READY')  # メッセージも初期化
            else:
                window['-TIMER_BTN-'].update(break_time_str)
                window['-MSG-'].update('BREAK TIME')
                break_time -= dt.timedelta(seconds=1)

window.close()

In [1]:
import PySimpleGUI as sg
import datetime as dt
import time

# 作業時間と休憩時間の設定
FOCUS_M, FOCUS_S = (0, 5)  # 作業時間（分、秒）
BREAK_M, BREAK_S = (0, 3)   # 休憩時間（分、秒）

# タイマーの初期化
def get_time(minutes, seconds):
    today = dt.datetime.today()
    return dt.datetime.combine(today, dt.time(0, minutes, seconds))

# タイマーのリセット機能
def reset_timer():
    focus_time = get_time(FOCUS_M, FOCUS_S)
    break_time = get_time(BREAK_M, BREAK_S)
    return focus_time, break_time

focus_time, break_time = reset_timer()

layout = [
    [sg.Button(f'{focus_time.strftime("%M:%S")}', font=('impact', 35), key='-TIMER_BTN-', size=(6, 1))],
    [sg.Text('READY', font=('impact', 15), key='-MSG-'), sg.Button('Reset')],
]

window = sg.Window('Pomodoro Timer', layout, size=(200, 150), location=(200, 200), element_justification='center')

timer_start = False
focus_start = True  # 最初は作業から開始

# イベントループ
while True:
    event, values = window.read(timeout=1000)  # 1秒ごとに更新
    if event == sg.WIN_CLOSED:
        break

    if event == 'Reset':
        timer_start = False
        focus_start = True
        focus_time, break_time = reset_timer()
        window['-TIMER_BTN-'].update(f'{focus_time.strftime("%M:%S")}')  # ボタンの表示も初期化
        window['-MSG-'].update('READY')  # メッセージも初期化

    if event == '-TIMER_BTN-':
        timer_start = not timer_start  # ボタンでスタート・ストップの切り替え

    if timer_start:
        if focus_start:
            focus_time_str = focus_time.strftime("%M:%S")
            if focus_time_str == "00:00":  # 修正点: 00:00に変更
                window['-TIMER_BTN-'].update(focus_time_str)
                focus_start = False  # 作業時間が終了したら休憩に切り替え
                focus_time = get_time(FOCUS_M, FOCUS_S)  # 作業時間をリセット
            else:
                window['-TIMER_BTN-'].update(focus_time_str)
                window['-MSG-'].update('FOCUS TIME')  # メッセージを更新
                focus_time -= dt.timedelta(seconds=1)
        else:
            break_time_str = break_time.strftime("%M:%S")
            if break_time_str == "00:00":  # 修正点: 00:00に変更
                timer_start = False  # 休憩時間が終了したらタイマーを停止して初期化
                focus_start = True  # 初期化して作業に切り替え
                focus_time, break_time = reset_timer()  # タイマーを初期状態にリセット
                window['-TIMER_BTN-'].update(f'{focus_time.strftime("%M:%S")}')  # ボタンの表示も初期化
                window['-MSG-'].update('READY')  # メッセージも初期化
            else:
                window['-TIMER_BTN-'].update(break_time_str)
                window['-MSG-'].update('BREAK TIME')  # メッセージを更新
                break_time -= dt.timedelta(seconds=1)

window.close()
